Test suite for testing matrix inv functions and the Kalman filter program

In the following cell: setting up
* imports 
* reading data
* functions to be tested


In [3]:
import numpy as np
import xarray as xr
#from blockinv_iterative import block_inv, block_iter_inv


ds_1 = xr.open_dataset("/home/pietaril/Documents/data/regions_verify_isotope_202112_cov.nc")
bio_1 = ds_1["covariance_bio"]
anth_1 = ds_1["covariance_anth"]

ds_2 = xr.open_dataset("/home/pietaril/Documents/data/regions_verify_202104_cov.nc")
bio_2 = ds_2["covariance_bio"]
anth_2 = ds_2["covariance_anth"]

M = bio_2.to_numpy() # test matrix as a numpy


In [4]:
ds_2

<xarray.Dataset>
Dimensions:           (nparams_bio: 6858, nparams_anth: 6858,
                       nparams_anth2: 6858)
Coordinates:
  * nparams_bio       (nparams_bio) int64 0 1 2 3 4 ... 6853 6854 6855 6856 6857
  * nparams_anth      (nparams_anth) int64 0 1 2 3 4 ... 6854 6855 6856 6857
  * nparams_anth2     (nparams_anth2) int64 0 1 2 3 4 ... 6854 6855 6856 6857
Data variables:
    covariance_bio    (nparams_bio, nparams_bio) float64 0.8 1.207e-08 ... 1e-08
    covariance_anth   (nparams_anth, nparams_anth) float64 ...
    covariance_anth2  (nparams_anth2, nparams_anth2) float64 ...

Simple matrix inv accuracy test:
* see if the inv function has calculated the inverse close enough.

In [35]:


inv1 = np.linalg.inv(M)
inv2 = block_inv(M,int(len(M)/2))
nv3 = block_iter_inv(M, 5)


print(np.allclose(inv1, inv2), np.allclose(inv1, inv3))

True True


Matrix inv timer 
* timing the execution of the inverse function on its own (not as part of the kf program)


In [36]:


%timeit np.linalg.inv(M)
%timeit block_inv(M,int(len(M)/2))
%timeit block_iter_inv(M,5)





5.09 s ± 121 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
6.41 s ± 1.64 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
7.91 s ± 231 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Tests for using thresholded matrix
* accuracy
* timer 


In [37]:
thresh = 10**(-15)

M0 = np.where(abs(M) < thresh, 0, M)

%timeit np.linalg.inv(M0)
%timeit block_inv(M0, 1500)
%timeit block_iter_inv(M0, 5)

LinAlgError: Singular matrix